# Create an Example Django Project

## Use django-admin to bootstrap the project

In [ ]:
# dont_test

%cd ..

/Users/jochen/projects/django_fileresponse


In [ ]:
# dont_test

!django-admin startproject example

/Users/jochen/projects/django_fileresponse


## Views

In [ ]:
%%writefile example/example/views.py
# dont_test

import aiofiles

from pathlib import Path

from django.http import HttpResponse
from django.http import FileResponse

from fileresponse.http import AiofileFileResponse


def get_file(request, num=None):
    file_path = Path(__file__).parent.parent / "data" / str(num)
    print(str(file_path))
    return FileResponse(file_path.open("rb"))


async def aget_file(request, num=None):
    file_path = Path(__file__).parent.parent / "data" / str(num)
    return AsyncFileResponse(file_path, chunk_size=1024)

Overwriting example/example/views.py


## Urls

In [ ]:
%%writefile example/example/urls.py
# dont_test

from django.urls import path

from . import views

urlpatterns = [
    path('<int:num>', views.get_file),
]

async_urlpatterns = [
    path('<int:num>', views.aget_file),
]

Overwriting example/example/urls.py


## ASGI

In [ ]:
%%writefile example/example/asgi.py
# dont_test
import os

import django
from django.core.asgi import get_asgi_application

from fileresponse.handlers import AsyncFileASGIHandler

from . import urls

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'example.settings')


def get_fileresponse_asgi_application():
    """
    The public interface to Django's ASGI support. Return an ASGI 3 callable.

    Avoids making django.core.handlers.ASGIHandler a public API, in case the
    internal implementation changes or moves in the future.
    """
    django.setup(set_prefix=False)
    urls.urlpatterns = urls.async_urlpatterns
    return AsyncFileASGIHandler()


# application = get_asgi_application()
application = get_fileresponse_asgi_application()

Overwriting example/example/asgi.py


## Apply Migrations

In [ ]:
# dont_test

%cd example
!python manage.py migrate

/Users/jochen/projects/django_fileresponse/example
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... 

## Link example to Notebooks Directory

In [ ]:
# dont_test

%cd ../nbs
!ln -s ../example/example .
%cd -

/Users/jochen/projects/django_fileresponse/nbs
ln: ./example: File exists
/Users/jochen/projects/django_fileresponse/example


## Start Server

Just run:
```shell
python manage.py runserver 0.0.0.0:8000
```